# Colbert

In [1]:
!git -C ColBERT/ pull || git clone https://github.com/stanford-futuredata/ColBERT.git
import sys; sys.path.insert(0, 'ColBERT/')

fatal: cannot change to 'ColBERT/': No such file or directory
Cloning into 'ColBERT'...
remote: Enumerating objects: 2835, done.
remote: Counting objects: 100% (1179/1179), done.
remote: Compressing objects: 100% (351/351), done.
remote: Total 2835 (delta 957), reused 829 (delta 828), pack-reused 1656 (from 2)
Receiving objects: 100% (2835/2835), 2.07 MiB | 20.56 MiB/s, done.
Resolving deltas: 100% (1788/1788), done.


## Si on run en dehors de collab regarder le readme (explique comment import sur conda)

In [2]:
try:
    import google.colab
    !pip install -U pip
    !pip install -e ColBERT/
    !pip install faiss-gpu torch
except Exception:
  import sys; sys.path.insert(0, 'ColBERT/')
  try:
    from colbert import Indexer, Searcher
  except Exception:
    print("If you're running outside Colab, please make sure you install ColBERT in conda following the instructions in our README. You can also install (as above) with pip but it may install slower or less stable faiss or torch dependencies. Conda is recommended.")
    assert False

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Obtaining file:///kaggle/working/ColBERT
  Preparing metadata (setup.py) ... done
  DEPRECATION: Legacy editable install of colbert-ai==0.2.20 from file:///kaggle/working/ColBERT (setup.py develop) is deprecated. pip 25.0 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457
  Running setup.py develop for colbert-ai
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 106.4 MB/s eta 0:00:0000:0100:01


In [3]:
import colbert
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

### Chargement dataset

In [4]:
from datasets import load_dataset, tqdm

In [5]:
ds_qa = load_dataset("enelpol/rag-mini-bioasq", "question-answer-passages")["test"]
ds_corpus = load_dataset("enelpol/rag-mini-bioasq", "text-corpus")["test"]

# only keep ds_qa rows with 3 items or more because we use 3 by default in our case
ds_qa = ds_qa.filter(lambda x: len(x["relevant_passage_ids"]) >= 3)

ds_qa.to_csv("../test_datasets/rag-mini-bioasq/qa/qa.csv")
ds_corpus.to_csv("../test_datasets/rag-mini-bioasq/corpus/corpus.csv")

README.md:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/187k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4012 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/707 [00:00<?, ? examples/s]

...

Generating test split:   0%|          | 0/40181 [00:00<?, ? examples/s]

Filter:   0%|          | 0/707 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/41 [00:00<?, ?ba/s]

60169807

In [6]:
import pandas as pd

qa_df = pd.read_csv("../test_datasets/rag-mini-bioasq/qa/qa.csv")
corpus_df = pd.read_csv("../test_datasets/rag-mini-bioasq/corpus/corpus.csv")

### Création de la collection

In [7]:
collection = corpus_df['passage'].tolist() 

### Création indexer

In [8]:
# Configuration pour l'indexation
checkpoint = 'colbert-ir/colbertv2.0'
index_name = 'rag_bioasq_index'
doc_maxlen = 300                            # Longueur maximale des documents
nbits = 2                                   # Encodage en 2 bits pour la compression

# Indexer les documents
#   nranks=1  :  nombre de machine sur laquel on execute
#   experiment='movie_plots' : nom de l'expérience
with Run().context(RunConfig(nranks=1, experiment='rag_bioasq')):
    print("Initialized Run context.")
    
    # Configuration Colbert (kmeans_niters = nb itération algo kmean)
    # Kmeans est utilisé pour regrouper les vecteurs en cluster => réduire la taille du stockage (car on stock que les centroides des cluster) et + rapide car on compare juste la distance entre centroïde et requète
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits, kmeans_niters=4)
    print("Configured ColBERT.")
    
    # Initialisation indexuer
    indexer = Indexer(checkpoint=checkpoint, config=config)
    print("Initialized Indexer.")
    
    # Indexer la collection
    indexer.index(name=index_name, collection=collection, overwrite=True)
    print("Indexing complete!")

Initialized Run context.
Configured ColBERT.


artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

Initialized Indexer.


[Jan 11, 16:48:59] #> Creating directory /kaggle/working/experiments/rag_bioasq/indexes/rag_bioasq_index 


#> Starting...
#> Joined...
Indexing complete!


# Calcul performance RAG

In [15]:
def test_retrieval(ds_qa, retrieve_function):
    total_num_documents_considered = 0
    num_valid_docs = 0
    
    for test_item in tqdm(ds_qa):
        question = test_item["question"]
        expected_documents_ids = test_item["relevant_passage_ids"]

        response = retrieve_function(question, k=3)

        ids = response[0]

        num_documents_considered = min(len(expected_documents_ids), len(ids))
        total_num_documents_considered += num_documents_considered

        for id in ids:
            if corpus_df['id'].iloc[id] in expected_documents_ids:
                num_valid_docs += 1     
        
    return num_valid_docs / total_num_documents_considered

In [14]:
from colbert import Searcher

with Run().context(RunConfig(experiment="rag_bioasq")):
    searcher = Searcher(index="rag_bioasq_index", collection=collection)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[Jan 11, 17:04:14] #> Loading codec...
[Jan 11, 17:04:14] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/kaggle/working/ColBERT/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
/kaggle/working/ColBERT/colbert/indexing/codecs/residual.py:141: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True`

[Jan 11, 17:04:14] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


...

[Jan 11, 17:04:14] #> Loading codes and residuals...



  0%|          | 0/2 [00:00<?, ?it/s]/kaggle/working/ColBERT/colbert/indexing/codecs/residual_embeddings.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.l

  0%|          | 0/497 [00:00<?, ?it/s]

/kaggle/working/ColBERT/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()



#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: Describe the mechanism of action of ibalizumab., 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  6235,  1996,  7337,  1997,  2895,  1997, 21307, 11475,
         9759,  2863,  2497,  1012,   102,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')



In [ ]:
score = test_retrieval(ds_qa, searcher.search)

In [16]:
print(f"RAG score: {score}")

RAG score: 0.7887323943661971


In [ ]:
!zip -r experiments/rag_bioasq/indexes/rag_bioasq_index experiments